## Import the libraries 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
contacts = pd.read_excel("Contacts.xlsx", header = 1)
applicants = pd.read_excel("Job Applicants.xlsx", header = 0)
jobs = pd.read_excel("Jobs.xlsx", header = 1)

## Identify the columns on which to merge

In [3]:
count = applicants[applicants['AVTRRT__Job__c'].isin(jobs['Id'])].shape[0]

# Stampa il conteggio delle osservazioni comuni
print("Osservazioni comuni:", count)

Osservazioni comuni: 71884


In [4]:
count2 = applicants[applicants['AVTRRT__Contact_Candidate__c'].isin(contacts['Id'])].shape[0]

# Stampa il conteggio delle osservazioni comuni
print("Osservazioni comuni:", count2)

Osservazioni comuni: 71091


## Creating column lists to eliminate duplicates

In [5]:
lista_app = applicants.columns.tolist()

In [6]:
lista_jobs = jobs.columns.tolist()

In [7]:
lista_contacts = contacts.columns.tolist()

In [8]:
set_app = set(lista_app)
set_jobs = set(lista_jobs)

common_items = list(set_app.intersection(set_jobs))

In [9]:
common_items.remove("Id")

In [10]:
jobs.drop(common_items, axis = 1, inplace = True)

In [11]:
jobs = jobs.rename(columns={'Id': 'AVTRRT__Job__c'})

## Merge applicants and jobs via the variables AVTRRT__Job__c and Id

In [12]:
merged_df = pd.merge(applicants, jobs, on = "AVTRRT__Job__c", how = "left")

## Same procedure for this new table and contacts

In [13]:
lista_merge = merged_df.columns.tolist()

In [14]:
set_merge = set(lista_merge)
set_contacts = set(lista_contacts)

In [15]:
common_items_2 = list(set_merge.intersection(set_contacts))

In [16]:
common_items_2.remove("Id")

In [17]:
contacts.drop(common_items_2, axis = 1, inplace = True)

In [18]:
contacts = contacts.rename(columns={'Id': 'AVTRRT__Contact_Candidate__c'})

In [19]:
merged_df_finale = pd.merge(merged_df, contacts, on = "AVTRRT__Contact_Candidate__c", how = "left")

## Do a first profiling to better understand the dataset

In [20]:
from ydata_profiling import ProfileReport

In [21]:
profile = ProfileReport(merged_df_finale, title="STEM HACKATON - Pandas Profiling Report", explorative=True)
profile.to_file("stem_hackaton.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

C:\Users\Fioruch\anaconda3\lib\site-packages\ydata_profiling\model\pandas\duplicates_pandas.py:37: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[duplicated_rows]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

KeyboardInterrupt: 

## Eliminate the variables that are not significant for our problem

In [22]:
merged_df_finale.drop([
    'IDnum',
    'OwnerId',
    'Name',
    'AVTRRT__Account_Job__c',
    'AVTRRT__Contact_Candidate__c',
    'AVTRRT__Job__c',
    'AVTRRT__Resume__c',  
    'AVTRRT__State__c',
    'Current_Role__c', #1/3 of the records are 00Eb0000000dAV4
    'Language_01__c', #too much nan
    'Language_02__c', #too much nan
    'Level_Language_01__c', #too much nan
    'Level_Language_02__c', #too much nan
    'AVTRRT__Job_Num__c',
    'AVTRRT__Closed_Qualifiers__c', ##with this feature we've seen what AVTRRTClosed_c values mean, but now we do not need it anymore
    'AVTRRT__Job_Country__c',
    'AVTRRT__Job_Description__c',
    'Location__c',
    'Travel__c',
    'Competitors__c',
    'Daily_Rate__c',
    'AVTRRT__Hired_Date__c',
    'AccountId',
    'RecordTypeId',
    'MailingCity',
    'MailingState',
    'MailingCountry',
    'AVTRRT__Candidate_Id__c',
    'AVTRRT__Candidate_Short_List__c',
    'AVTRRT__Candidate_Summary__c',
    'AVTRRT__Candidate_Write_Up__c',
    'AVTRRT__Cover_Letter__c',
    'AVTRRT__Current_Pay__c',
    'AVTRRT__Education_Details__c',
    'AVTRRT__General_Competency__c',
    'AVTRRT__IT_Competency__c',
    'AVTRRT__Other_Competency__c',
    ' ',
    'AVTRRT__Previous_Titles__c',
    'Language_03__c',
    'Level_Language_03__c',
    'Desired_companies__c',
    'Bonus__c',
    'Tot_Experience__c',
    'Unnamed: 55',
    'P_L_Responsability__c'
], axis=1, inplace = True)

In [23]:
df_finale = merged_df_finale.copy() 

## Analyze the NAs

In [24]:
from termcolor import colored

In [25]:
def missing_values_percentage(df):
    total_rows = df.shape[0]
    columns_with_high_missing_values = []
    for column in df.columns:
        missing_count = df[column].isnull().sum()
        missing_percentage = (missing_count / total_rows) * 100
        if missing_percentage > 40:
            print(colored(f'Column "{column}" has {missing_percentage:.2f}% missing values.', 'red'))
            columns_with_high_missing_values.append(column)
        else:
            print(f'Column "{column}" has {missing_percentage:.2f}% missing values.')

    return columns_with_high_missing_values


columns_with_high_missing_values = missing_values_percentage(df_finale)
print("Columns with high missing values:", columns_with_high_missing_values)

Column "Id" has 0.00% missing values.
Column "AVTRRT__City__c" has 35.40% missing values.
Column "AVTRRT__Desired_Pay_Range_From__c" has 72.50% missing values.
Column "AVTRRT__Experience__c" has 99.52% missing values.
Column "AVTRRT__Job_Title__c" has 0.00% missing values.
Column "AVTRRT__Pay_Type__c" has 71.24% missing values.
Column "AVTRRT__Percentage_of_Travel__c" has 75.61% missing values.
Column "AVTRRT__Relocation__c" has 0.00% missing values.
Column "AVTRRT__Stage__c" has 26.76% missing values.
Column "AVTRRT__Type__c" has 0.00% missing values.
Column "Relevant_Qualification__c" has 98.60% missing values.
Column "Reason_for_leaving__c" has 98.41% missing values.
Column "Overall_Experience__c" has 97.43% missing values.
Column "Specific_Experience__c" has 97.43% missing values.
Column "Current_Salary__c" has 99.89% missing values.
Column "Current_Bonus__c" has 99.95% missing values.
Column "Current_Benefits__c" has 99.97% missing values.
Column "Notice_Period__c" has 97.75% miss

## Eliminate columns consisting mainly of NAs

In [26]:
#high_missing_values
high_missing = ['AVTRRT__Desired_Pay_Range_From__c',
 'AVTRRT__Experience__c',
'AVTRRT__Open_For_Relocation__c',
 'AVTRRT__Pay_Type__c',
 'AVTRRT__Percentage_of_Travel__c',
 'Relevant_Qualification__c',
 'Reason_for_leaving__c',
 'Overall_Experience__c',
 'Specific_Experience__c',
 'Current_Salary__c',
 'Current_Bonus__c',
 'Current_Benefits__c',
 'Notice_Period__c',
 'Contract_Level__c',
 'Team_Leading_Responsibility__c',
 'Profit_and_Loss_Resp__c',
 'International_experience__c',
 'Type_of_Studies__c',
 'Non_Compete_Agreements__c',
 'Field_of_Studies__c',
 'Year_of_Birth__c',
 'AVTRRT__General_Job_Category_Sought__c',
 'AVTRRT__Locations_1st_Choice__c',
 'Area_of_interest_b__c',
 'Reason_for_Leaving__c',
 'Type_of_Experience__c',
 'P_L_Responsibility__c',
 'International_Experience__c',
 'CCNL__c',
 'Specialisation_Industry__c',
 'CCNL_Level__c',
'Relocation__c',
"AVTRRT__Closed__c"]
df_finale.drop(high_missing, axis = 1, inplace = True)

## Analyse the correlation between variables and decide to eliminate Open to relocation (in the previous chunk)

In [27]:
import seaborn as sns

In [28]:
correlation_matrix = df_finale.corr()
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='RdYlGn', linewidths=0.5)
plt.title('Heatmap di Correlazione')
plt.show()

C:\Users\Fioruch\AppData\Local\Temp\ipykernel_8432\1115125071.py:5: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  plt.show()


 ## Renaming columns to avoid double underscore problems

In [29]:
df_finale.rename(columns=lambda x: x.replace('__', '_'), inplace=True)

## Create a copy on which to work on preprocessing for modelling and do another check with profiling

In [30]:
df = df_finale.copy()

In [36]:
profile_fin = ProfileReport(df, title="STEM HACKATON FINALE - Pandas Profiling Report", explorative=True)
profile_fin.to_file("stem_hackaton_fin.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

## Check that the index is right after the various drops

In [31]:
df.index = range(1, len(df) + 1)

In [32]:
df.Id[52701]  == 'a0Db000000V3XVjEAN' #check for index

True

## Divide in train and test set

In [33]:
train = df[:52700]

In [34]:
test = df[52700:]

In [35]:
train = train[train['AVTRRT_Stage_c'] != 2]

## Creating a function to assess the performance of upcoming models

In [36]:
from sklearn.metrics import classification_report
from yellowbrick.classifier import ConfusionMatrix
from yellowbrick.classifier import ClassificationReport
from yellowbrick.classifier import ROCAUC
from yellowbrick.classifier import PrecisionRecallCurve
from yellowbrick.classifier import ClassPredictionError
from yellowbrick.classifier import DiscriminationThreshold

In [37]:
from yellowbrick.exceptions import YellowbrickTypeError

def classification_results(model, X_train, y_train, X_val, y_val, classes=["non assunto", "assunto"]):
    classes = classes

    # sklearn classification_report
    cr = classification_report(y_val, model.predict(X_val), target_names=classes)
    print(cr)

    if isinstance(model, CatBoostClassifier):
        print("Skipping Yellowbrick visualizations for CatBoostClassifier.")
        return

    try:
        from yellowbrick.classifier import ClassificationReport
        cr = ClassificationReport(model, classes=classes)
        cr.score(X_val, y_val)
        cr.show()
    except YellowbrickTypeError as e:
        print("YellowbrickTypeError caught", e)

    try:
        from yellowbrick.classifier import ConfusionMatrix
        cm = ConfusionMatrix(model, classes=classes)
        cm.score(X_val, y_val)
        cm.show()
    except YellowbrickTypeError as e:
        print("YellowbrickTypeError caught", e)

    try:
        from yellowbrick.classifier import ClassPredictionError
        pe = ClassPredictionError(model, classes=classes)
        pe.score(X_val, y_val)
        pe.show()
    except YellowbrickTypeError as e:
        print("YellowbrickTypeError caught", e)

    try:
        from yellowbrick.classifier import ROCAUC
        ra = ROCAUC(model, classes=classes)
        ra.score(X_val, y_val)
        ra.show()
    except YellowbrickTypeError as e:
        print("YellowbrickTypeError caught", e)

    try:
        from yellowbrick.classifier import PrecisionRecallCurve
        pr = PrecisionRecallCurve(model)
        pr.score(X_val, y_val)
        pr.show()
    except YellowbrickTypeError as e:
        print("YellowbrickTypeError caught", e)


In [38]:
y_cat = train["AVTRRT_Stage_c"]
X_cat= train.drop(["AVTRRT_Stage_c", "Id"], axis = 1)

cat_features = X_cat.select_dtypes(include="object").columns.tolist()

num_features = X_cat.select_dtypes(include=["float64", "int64"]).columns.tolist()
X_cat[cat_features] = X_cat[cat_features].fillna('unknown')
column_means = X_cat[num_features].mean()

# Riempimento dei valori mancanti con la media delle colonne
X_cat[num_features] = X_cat[num_features].fillna(column_means)

In [39]:
from sklearn.model_selection import train_test_split

In [40]:
X_train_cat, X_val_cat, y_train_cat, y_val_cat = train_test_split(X_cat, y_cat, test_size=0.2, stratify = y_cat)

# Catboost 

In [41]:
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score

In [42]:
model_CBC = CatBoostClassifier(iterations=4000, loss_function = 'CrossEntropy', cat_features= cat_features)
model_CBC.fit(X_train_cat, y_train_cat.ravel())
print(model_CBC)

0:	learn: 0.6655857	total: 278ms	remaining: 18m 33s
1:	learn: 0.6390241	total: 370ms	remaining: 12m 19s
2:	learn: 0.6135236	total: 456ms	remaining: 10m 7s
3:	learn: 0.5891669	total: 536ms	remaining: 8m 55s
4:	learn: 0.5691732	total: 589ms	remaining: 7m 50s
5:	learn: 0.5477289	total: 674ms	remaining: 7m 28s
6:	learn: 0.5291403	total: 749ms	remaining: 7m 6s
7:	learn: 0.5126330	total: 834ms	remaining: 6m 55s
8:	learn: 0.4966091	total: 906ms	remaining: 6m 41s
9:	learn: 0.4811939	total: 985ms	remaining: 6m 33s
10:	learn: 0.4663541	total: 1.07s	remaining: 6m 29s
11:	learn: 0.4525291	total: 1.15s	remaining: 6m 22s
12:	learn: 0.4393983	total: 1.22s	remaining: 6m 13s
13:	learn: 0.4277112	total: 1.3s	remaining: 6m 9s
14:	learn: 0.4164743	total: 1.38s	remaining: 6m 5s
15:	learn: 0.4072635	total: 1.45s	remaining: 5m 59s
16:	learn: 0.3990457	total: 1.49s	remaining: 5m 49s
17:	learn: 0.3913916	total: 1.56s	remaining: 5m 44s
18:	learn: 0.3840883	total: 1.63s	remaining: 5m 41s
19:	learn: 0.3769351	tot

KeyboardInterrupt: 

In [95]:
y_pred_cat = model_CBC.predict(X_val_cat)

In [96]:
accuracy_score(y_val_cat, y_pred_cat)

0.9082125603864735

In [ ]:
model_CBC = CatBoostClassifier(iterations=4000, loss_function = 'CrossEntropy', cat_features= cat_features)
model_CBC.fit(X_cat, y_cat.ravel())
print(model_CBC)

In [148]:
y_test_cat = test["AVTRRT_Stage_c"]
id_cat = test["Id"]
X_test_cat = test.drop(["AVTRRT_Stage_c", "Id"], axis = 1)

In [149]:
X_test_cat[cat_features] = X_test_cat[cat_features].fillna('unknown')
column_means = X_test_cat[num_features].median()

# Riempimento dei valori mancanti con la media delle colonne
X_test_cat[num_features] = X_test_cat[num_features].fillna(column_means)

In [150]:
y_pred_test_cat = model_CBC.predict(X_test_cat)

In [ ]:
data_cat = pd.DataFrame({'Id': id_cat, 'Predictions': y_pred_test_cat})

In [152]:
data_cat.to_csv("pred.csv", index = False, header = False)

In [ ]:
pd.read_csv('pred.csv').drop('a0Db000000V3XVjEAN', axis=1).to_csv('preds_one_column.csv', index=False)

## Data preprocessing

In [43]:
y = train["AVTRRT_Stage_c"]
X = train.drop(["AVTRRT_Stage_c", "Id"], axis = 1)

In [44]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify = y_cat)

In [45]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from category_encoders import LeaveOneOutEncoder, OrdinalEncoder

In [46]:
num_pipe = Pipeline([
    ('imputer', SimpleImputer(missing_values=np.nan, strategy='median'))
])

leave_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='Unknown')),
    ('transformer', LeaveOneOutEncoder())
])

hot_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='Unknown')),
    ('encoder', OneHotEncoder(drop = 'first', sparse = False))
])

ord_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='Unknown')),
    ('encoder', OrdinalEncoder([['Unknown', '0', '1-2', '3-5', '6-10', '10']]))
])

In [47]:
from sklearn.compose import ColumnTransformer

In [ ]:
ct = ColumnTransformer([
    ('num', num_pipe, num_features),
    ('leave', leave_pipe, ['AVTRRT_City_c', 'AVTRRT_Job_Title_c', 'AVTRRT_Job_Category_c','AVTRRT_Job_Term_c',
                                              'Job_Type_c', 'OSN_Practice_Focus_c']),
    ('hot', hot_pipe, ['Job_Type_c', 'Type_of_contract_c', 'Project_Management_c', 'Type_of_contract_c']),
    ('ord', ord_pipe, ['Team_Leading_c']),
],
    remainder='drop'
)
ct

## Logistic Regression

In [54]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()

In [55]:
pipe = Pipeline([
    ('ct', ct),
    ('classifier', model)
])
pipe

Pipeline(steps=[('ct',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  ['AVTRRT_Relocation_c',
                                                   'AVTRRT_Number_of_Applicants_c',
                                                   'AVTRRT_Number_of_Positions_c',
                                                   'AVTRRT_Number_of_Interviews_c',
                                                   'Fix_Salary_c',
                                                   'Variable_Bonus_c',
                                                   'Commission_Fee_c']),
                                                 ('leave',
                                                  Pipeline(steps=[('imputer',
                                                                 

In [56]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'classifier__penalty': ['l2'],
    'classifier__C': [0.1, 1.0, 10.0]
}

grid = GridSearchCV(
    estimator=pipe,
    param_grid=param_grid,
    cv=3,
    verbose=3, 
    n_jobs=-1, 
    #pre_dispatch=4,
    return_train_score=True,
)

model_grid = grid.fit(X_train, y_train.ravel())

Fitting 3 folds for each of 3 candidates, totalling 9 fits


In [57]:
model_grid.best_params_

{'classifier__C': 0.1, 'classifier__penalty': 'l2'}

In [58]:
model = model_grid.best_estimator_

In [59]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

confusion_matrix(y_val, model.predict(X_val))

array([[  23, 1518],
       [  67, 5637]], dtype=int64)

In [60]:
print(classification_report(y_val, model.predict(X_val)))

              precision    recall  f1-score   support

         0.0       0.26      0.01      0.03      1541
         1.0       0.79      0.99      0.88      5704

    accuracy                           0.78      7245
   macro avg       0.52      0.50      0.45      7245
weighted avg       0.67      0.78      0.70      7245



## lightGbm

In [63]:
from lightgbm import LGBMClassifier

model = LGBMClassifier()

In [64]:
pipe = Pipeline([
    ('ct', ct),
    ('classifier', model)
])
pipe

Pipeline(steps=[('ct',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  ['AVTRRT_Relocation_c',
                                                   'AVTRRT_Number_of_Applicants_c',
                                                   'AVTRRT_Number_of_Positions_c',
                                                   'AVTRRT_Number_of_Interviews_c',
                                                   'Fix_Salary_c',
                                                   'Variable_Bonus_c',
                                                   'Commission_Fee_c']),
                                                 ('leave',
                                                  Pipeline(steps=[('imputer',
                                                                 

In [70]:
pipe.fit(X_train, y_train.ravel())

Pipeline(steps=[('ct',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  ['AVTRRT_Relocation_c',
                                                   'AVTRRT_Number_of_Applicants_c',
                                                   'AVTRRT_Number_of_Positions_c',
                                                   'AVTRRT_Number_of_Interviews_c',
                                                   'Fix_Salary_c',
                                                   'Variable_Bonus_c',
                                                   'Commission_Fee_c']),
                                                 ('leave',
                                                  Pipeline(steps=[('imputer',
                                                                 

In [72]:
confusion_matrix(y_val, pipe.predict(X_val))

array([[   0, 1541],
       [   0, 5704]], dtype=int64)

In [73]:
print(classification_report(y_val, pipe.predict(X_val)))

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00      1541
         1.0       0.79      1.00      0.88      5704

    accuracy                           0.79      7245
   macro avg       0.39      0.50      0.44      7245
weighted avg       0.62      0.79      0.69      7245



C:\Users\Fioruch\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Fioruch\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Fioruch\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Xgboost

In [81]:
from xgboost import XGBClassifier

model = XGBClassifier()

In [82]:
pipe = Pipeline([
    ('ct', ct),
    ('classifier', model)
])
pipe

Pipeline(steps=[('ct',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  ['AVTRRT_Relocation_c',
                                                   'AVTRRT_Number_of_Applicants_c',
                                                   'AVTRRT_Number_of_Positions_c',
                                                   'AVTRRT_Number_of_Interviews_c',
                                                   'Fix_Salary_c',
                                                   'Variable_Bonus_c',
                                                   'Commission_Fee_c']),
                                                 ('leave',
                                                  Pipeline(steps=[('imputer',
                                                                 

In [83]:
pipe.fit(X_train, y_train.ravel())

Pipeline(steps=[('ct',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  ['AVTRRT_Relocation_c',
                                                   'AVTRRT_Number_of_Applicants_c',
                                                   'AVTRRT_Number_of_Positions_c',
                                                   'AVTRRT_Number_of_Interviews_c',
                                                   'Fix_Salary_c',
                                                   'Variable_Bonus_c',
                                                   'Commission_Fee_c']),
                                                 ('leave',
                                                  Pipeline(steps=[('imputer',
                                                                 

In [84]:
confusion_matrix(y_val, pipe.predict(X_val))

array([[   1, 1540],
       [   0, 5704]], dtype=int64)

In [85]:
print(classification_report(y_val, pipe.predict(X_val)))

              precision    recall  f1-score   support

         0.0       1.00      0.00      0.00      1541
         1.0       0.79      1.00      0.88      5704

    accuracy                           0.79      7245
   macro avg       0.89      0.50      0.44      7245
weighted avg       0.83      0.79      0.69      7245



## Conclusion

In the process of choosing a suitable model, we faced a decision between CatBoost, which offers automatic preprocessing and high performance, and a variety of other models that, despite exhaustive preprocessing, failed to yield comparable results. Ultimately, we selected CatBoost as our model of choice, driven by its superior accuracy and recall metrics.